In [5]:
library(tidyr)
library(dplyr)
library(lubridate)
library(xts)

completeHvalerFile = "Hvaler/complete.csv"
completeHvalerClasses = c('POSIXct', rep("numeric", 21))
HvalerDf = read.csv(completeHvalerFile, stringsAsFactors=FALSE, colClasses=completeHvalerClasses)
xt = xts(x = HvalerDf[, -1], order.by = HvalerDf[, 1])

In [6]:
#Impute temperature first (Hvaler has 9 missing values on temperature)
require("imputeTS")
imputed_xt = na.kalman(as.ts(xt$T01), model = "auto.arima", smooth = TRUE)
imputed_xts = xts(imputed_xt, index(xt))
HvalerDf[match(index(imputed_xts), HvalerDf$DateTime), "T01"] = coredata(imputed_xts)

In [28]:
#str(imputedDf)
#imputed_DF = rfImpute(T01 ~ year + month + weekend + hour+ subs.1, imputedDf, ntree = 100)
library(Amelia)
df = HvalerDf %>% mutate(hour_weekend = as.factor(paste0(as.character(hour(DateTime)),
                                                         ".",
                                                         as.character(wday(DateTime) %in% c(1, 7)))))
cols = seq(2, 21)
min = rep(0, 20)
max = rep(9999999, 20)
bounds = as.matrix(cbind(cols, min, max))

out = amelia(df, ts = "T01", polytime = 2, cs = 'hour_weekend', intercs = TRUE, bounds=bounds, max.resample = 1000)
str(out)


-- Imputation 1 --

  1  2  3  4  5  6  7  8

-- Imputation 2 --

  1  2  3  4  5  6  7  8

-- Imputation 3 --

  1  2  3  4  5  6  7  8  9 10 11

-- Imputation 4 --

  1  2  3  4  5  6  7  8

-- Imputation 5 --

  1  2  3  4  5  6  7  8  9

List of 12
 $ imputations:List of 5
  ..$ imp1:'data.frame':	17544 obs. of  23 variables:
  .. ..$ DateTime    : POSIXct[1:17544], format: "2012-01-01 00:00:00" "2012-01-01 01:00:00" ...
  .. ..$ subs.1      : num [1:17544] 253 228 214 229 212 ...
  .. ..$ subs.2      : num [1:17544] 8.232 16.891 35.914 0.308 10.867 ...
  .. ..$ subs.3      : num [1:17544] 202 174 189 182 148 ...
  .. ..$ subs.4      : num [1:17544] 62.8 57.9 59.2 58.9 61 ...
  .. ..$ subs.5      : num [1:17544] 40.7 41.1 41.4 37.7 38.3 ...
  .. ..$ subs.6      : num [1:17544] 76.4 73.9 74.5 71.4 70 ...
  .. ..$ subs.7      : num [1:17544] 72.5 72.3 70.1 68.2 66.4 ...
  .. ..$ subs.8      : num [1:17544] 47.5 42.1 42.6 45.5 39.7 ...
  .. ..$ subs.9      : num [1:17544] 146 154 152 

In [29]:
imputedHvalerDf = out$imputations$imp1 %>% select (-hour_weekend)
write.csv(imputedHvalerDf, "Hvaler/imputed_complete.csv", row.names=FALSE)

#Draw imputed dataset
source("http://www.staff.uni-marburg.de/~appelhat/r_stuff/strip.R")

for (i in seq(1, 20)){
    zone = paste0("subs.", i);
    pdf(paste0("imputed_", zone, "_Hvaler_strip.pdf"),width=7,height=5)
    zoneDf = out$imputations$imp1
    strip(x = zoneDf[[zone]], 
      date = zoneDf$DateTime,
      cond = year(zoneDf$DateTime),
      arrange = "wide",
      main = paste("Consumption", zone))
    dev.off()
}



 Module   :  strip 
 Author   :  Tim Appelhans <tim.appelhans@gmail.com>, Thomas Nauss 
 Version  :  2012-01-06 
 License  :  GNU GPLv3, see http://www.gnu.org/licenses/ 

 Module   :  strip 
 Author   :  Tim Appelhans <tim.appelhans@gmail.com>, Thomas Nauss 
 Version  :  2012-01-06 
 License  :  GNU GPLv3, see http://www.gnu.org/licenses/ 

 Module   :  strip 
 Author   :  Tim Appelhans <tim.appelhans@gmail.com>, Thomas Nauss 
 Version  :  2012-01-06 
 License  :  GNU GPLv3, see http://www.gnu.org/licenses/ 

 Module   :  strip 
 Author   :  Tim Appelhans <tim.appelhans@gmail.com>, Thomas Nauss 
 Version  :  2012-01-06 
 License  :  GNU GPLv3, see http://www.gnu.org/licenses/ 

 Module   :  strip 
 Author   :  Tim Appelhans <tim.appelhans@gmail.com>, Thomas Nauss 
 Version  :  2012-01-06 
 License  :  GNU GPLv3, see http://www.gnu.org/licenses/ 

 Module   :  strip 
 Author   :  Tim Appelhans <tim.appelhans@gmail.com>, Thomas Nauss 
 Version  :  2012-01-06 
 License  :  GNU GPLv3, se

In [7]:
#library(randomForest)
#imputedDf = HvalerDf %>% mutate(year=year(DateTime), month=month(DateTime), 
#                                weekend=wday(DateTime) %in% c(1, 7), hour=hour(DateTime))%>%
#                         mutate(weekend = as.factor(weekend), hour = as.numeric(hour)) %>%
#                         select(-DateTime)
